## Linux Flight Comptuer

We also had some simple monitor code running on the main flight comptuer itself. It's an Intel Atom processor running more-or-less off-the-shelf linux (Debian).

We can look at the CPU usage. These numbers come from a python utility running on the flight comptuer using the `psutil` package. Health stat program can be found here:

<https://github.com/psas/av3-fc/tree/master/health-monitor>

From the psutil docs:

> `psutil.cpu_times(percpu=False)` Return system CPU times as a namedtuple. Every attribute represents the seconds the CPU has spent in the given mode.

We look at the modes `user`, `sys`, `IOwait`, `IRQ`, and `softIRQ`.

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var, average, arange, absolute, insert
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

t_0 = 117853569585227

columns = loadtxt("../fc-data/FCFH.csv", delimiter=',', unpack=True)

seqn = columns[0]
timestamp = columns[1]
cpu_user, cpu_sys, cpu_nice, cpu_IOwait, cpu_IRQ, cpu_softIRQ = columns[2:8]
ram_used, ram_buffer, ram_cached = columns[8:11]
PIDs = columns[11]
disk_used, dsk_read, disk_Write = columns[12:15]
IO_lo_Bytes_Sent, IO_lo_Bytes_Recv, IO_lo_Packets_Sent, IO_lo_Packets_Recv = columns[15:19]
IO_eth0_Bytes_Sent, IO_eth0_Bytes_Recv, IO_eth0_Packets_Sent, IO_eth0_Packets_Recv = columns[19:23]
IO_wlan0_Bytes_Sent, IO_wlan0_Bytes_Recv, IO_wlan0_Packets_Sent, IO_wlan0_Packets_Recv = columns[23:27]
core_temp = columns[27]

timestamp = subtract(timestamp, t_0)
timestamp = divide(timestamp, 1e9)


fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Main Flight Comptuer CPU Usage By Mode")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"CPU Use [%]")
ax1.fill_between(timestamp, cpu_IRQ+cpu_softIRQ+cpu_IOwait+cpu_sys+cpu_user, cpu_IRQ+cpu_softIRQ+cpu_sys+cpu_user, alpha=0.8, color="r", label="I0 Wait")
ax1.fill_between(timestamp, cpu_IRQ+cpu_softIRQ+cpu_sys+cpu_user, cpu_sys+cpu_user, alpha=0.4, label="IRQ")
ax1.fill_between(timestamp, cpu_sys+cpu_user, cpu_user, alpha=0.5, color="#709afa", label="System")
ax1.fill_between(timestamp, cpu_user, 0, alpha=0.5, color="#76e146", label="User")
ax1.set_xlim([-5,42])
ax1.set_ylim([0,100])
ax1.legend(loc=2)
plt.show()

We notice that IO Wait times are suddently huge for brief periods during the flight. In hindsight, it would have been nice to have higher time resolution (this was recoreded at 1 Hz). Maybe should 10 Hz in the future.

It's possible that the this corresponds to the data gaps. We can overlay the timing of data gaps with the cpu chart. This might indicate a problem somewhere in getting data.

In [ ]:
columns = loadtxt("../fc-data/SEQE.csv", delimiter=',', unpack=True)

seqn      = columns[0]
seqe_time = columns[1]
seqe_time = subtract(seqe_time, t_0)
seqe_time = divide(seqe_time, 1e9)


fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Main Flight Comptuer CPU Usage By Mode")
plt.xlabel(r"Mission Elapsed Time [s]")
plt.ylabel(r"CPU Use [%]")
ax1.fill_between(timestamp, cpu_IRQ+cpu_softIRQ+cpu_IOwait+cpu_sys+cpu_user, cpu_IRQ+cpu_softIRQ+cpu_sys+cpu_user, alpha=0.8, color="r", label="I0 Wait")
ax1.fill_between(timestamp, cpu_IRQ+cpu_softIRQ+cpu_sys+cpu_user, cpu_sys+cpu_user, alpha=0.4, label="IRQ")
ax1.fill_between(timestamp, cpu_sys+cpu_user, cpu_user, alpha=0.5, color="#709afa", label="System")
ax1.fill_between(timestamp, cpu_user, 0, alpha=0.5, color="#76e146", label="User")

for t in seqe_time[1:]:
    plt.plot((t, t), (0, 100), 'k-', alpha=0.7)
plt.plot((-50, -50), (0, 100), 'k-', alpha=0.7, label="Data Gaps")

ax1.set_xlim([-5,42])
ax1.set_ylim([0,100])
ax1.legend(loc=2)
plt.show()